In [257]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from memory_profiler import memory_usage
import tracemalloc
data=pd.read_csv("Life_Expectancy_Data.csv")
data=data.dropna()

In [178]:

data1 = data[data['Year'] <= 2010]
data2 = data[data['Year'] > 2010]
data1=data1.replace("Developed",1)
data1=data1.replace("Developing",0)
data2=data2.replace("Developed",1)
data2=data2.replace("Developing",0)
yinput=data1[data1.columns[2]].values
xinput=data1[data1.columns[3:8]].values
youtput=data2[data2.columns[2]].values
xoutput=data2[data2.columns[3:8]].values

In [265]:

from memory_profiler import memory_usage
random.seed(1)

# Start tracing memory usage
tracemalloc.start()
# Define MLP model architecture using TensorFlow
model = tf.keras.Sequential([
    tf.keras.layers.Dense(5, activation='sigmoid', input_shape=(5,)),
    tf.keras.layers.Dense(3, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model with binary cross-entropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer='Adam')

# Train the model using the input data and target labels

model.fit(xinput, yinput, epochs=1000, verbose=1)

current, peak = tracemalloc.get_traced_memory()
print(f"Peak memory usage: {peak / 10**6} MB")

# Stop tracing memory usage
tracemalloc.stop()



Epoch 1/1000
36/36 [==============================] - 4s 4ms/step - loss: 0.4947
Epoch 2/1000
36/36 [==============================] - 0s 5ms/step - loss: 0.4637
Epoch 3/1000
36/36 [==============================] - 0s 5ms/step - loss: 0.4390
Epoch 4/1000
36/36 [==============================] - 0s 5ms/step - loss: 0.4171
Epoch 5/1000
36/36 [==============================] - 0s 4ms/step - loss: 0.4004
Epoch 6/1000
36/36 [==============================] - 0s 5ms/step - loss: 0.3881
Epoch 7/1000
36/36 [==============================] - 0s 4ms/step - loss: 0.3777
Epoch 8/1000
36/36 [==============================] - 0s 4ms/step - loss: 0.3687
Epoch 9/1000
36/36 [==============================] - 0s 4ms/step - loss: 0.3610
Epoch 10/1000
36/36 [==============================] - 0s 3ms/step - loss: 0.3541
Epoch 11/1000
36/36 [==============================] - 0s 4ms/step - loss: 0.3479
Epoch 12/1000
36/36 [==============================] - 0s 4ms/step - loss: 0.3427
Epoch 13/1000
36/36 [====

In [266]:
random.seed(1)
# Test the model by predicting the target labels for new input data
predictions = model.predict(np.array(xoutput))
for i in range(len(youtput)):
  if predictions[i]>=0.6:
    predictions[i]=1
  else:
    predictions[i]=0

17/17 [==============================] - 0s 1ms/step


In [267]:
predictions=predictions.ravel().tolist()

In [268]:
count1 = np.count_nonzero(predictions == youtput)
count1

475

In [281]:
import torch
import torch.nn as nn
import numpy as np

# Start tracing memory usage
tracemalloc.start()

random.seed(1)
# Define the input and output data
x = np.array(xinput,dtype=np.float32)
y = np.array(yinput,dtype=np.float32)

# Define the PyTorch MLP model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(5, 5)  # First fully connected layer with 3 input features and 4 output features
        self.fc2 = nn.Linear(5, 2)  # Second fully connected layer with 4 input features and 2 output features
        self.relu = nn.ReLU()  # ReLU activation function

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        return out

# Define the loss function and optimizer
model1 = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model1.parameters(), lr=0.01)

# Train the model
num_epochs = 1000
for epoch in range(num_epochs):
    inputs = torch.from_numpy(x)
    labels = torch.from_numpy(y)

    # Forward pass
    outputs = model1(inputs)
    loss = criterion(outputs, labels.long())

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# Test the model
with torch.no_grad():
    inputs = torch.from_numpy(np.array(xoutput,dtype=np.float32))
    outputs = model1(inputs)
    _, predicted = torch.max(outputs.data, 1)
    print('Predicted:', predicted.numpy())

current, peak = tracemalloc.get_traced_memory()
print(f"Peak memory usage: {peak / 10**6} MB")

# Stop tracing memory usage
tracemalloc.stop()


Epoch [100/1000], Loss: 0.2375
Epoch [200/1000], Loss: 0.1825
Epoch [300/1000], Loss: 0.1545
Epoch [400/1000], Loss: 0.2902
Epoch [500/1000], Loss: 0.1536
Epoch [600/1000], Loss: 0.1472
Epoch [700/1000], Loss: 0.1447
Epoch [800/1000], Loss: 0.1433
Epoch [900/1000], Loss: 0.1440
Epoch [1000/1000], Loss: 0.1428
Predicted: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0
 0 0 0 0 0 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 1 0 1 1 1 1 0
 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 

In [282]:
count2 = np.count_nonzero(predicted.numpy() == youtput)
count2

492

In [301]:
yinput1=data1[data1.columns[3]].values
xinput1=data1[data1.columns[2:8]]
xinput1=xinput1.drop(xinput1.columns[1],axis=1).values
youtput1=data2[data2.columns[3]].values
xoutput1=data2[data2.columns[2:8]]
xoutput1=xoutput1.drop(xoutput1.columns[1],axis=1).values


In [315]:
from memory_profiler import memory_usage
random.seed(1)

# Start tracing memory usage
tracemalloc.start()
# Define MLP model architecture using TensorFlow
model2 = tf.keras.Sequential([
    tf.keras.layers.Dense(5, activation='sigmoid', input_shape=(5,)),
    tf.keras.layers.Dense(3, activation='relu'),
    tf.keras.layers.Dense(1, activation='relu')
])

# Compile the model with binary cross-entropy loss and Adam optimizer
model2.compile(loss='binary_crossentropy', optimizer='Adam')

# Train the model using the input data and target labels

model2.fit(xinput1, yinput1, epochs=1000, verbose=1)

current, peak = tracemalloc.get_traced_memory()
print(f"Peak memory usage: {peak / 10**6} MB")

# Stop tracing memory usage
tracemalloc.stop()


Epoch 1/1000
36/36 [==============================] - 2s 3ms/step - loss: 1062.1509
Epoch 2/1000
36/36 [==============================] - 0s 3ms/step - loss: 1062.1506
Epoch 3/1000
36/36 [==============================] - 0s 3ms/step - loss: 1062.1509
Epoch 4/1000
36/36 [==============================] - 0s 3ms/step - loss: 1062.1508
Epoch 5/1000
36/36 [==============================] - 0s 4ms/step - loss: 1062.1508
Epoch 6/1000
36/36 [==============================] - 0s 4ms/step - loss: 1062.1508
Epoch 7/1000
36/36 [==============================] - 0s 4ms/step - loss: 1062.1506
Epoch 8/1000
36/36 [==============================] - 0s 3ms/step - loss: 1062.1506
Epoch 9/1000
36/36 [==============================] - 0s 3ms/step - loss: 1062.1508
Epoch 10/1000
36/36 [==============================] - 0s 3ms/step - loss: 1062.1509
Epoch 11/1000
36/36 [==============================] - 0s 3ms/step - loss: 1062.1508
Epoch 12/1000
36/36 [==============================] - 0s 3ms/step - loss:

In [316]:
random.seed(1)
# Test the model by predicting the target labels for new input data
predictions1 = model2.predict(np.array(xoutput1))
predictions1=predictions1.ravel().tolist()

17/17 [==============================] - 0s 2ms/step


In [317]:
yinput1

array([58.8, 58.6, 58.1, ..., 44.8, 45.3, 46. ])

In [318]:
predictions1

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [325]:
# Start tracing memory usage
tracemalloc.start()

random.seed(1)
# Define the input and output data
x1 = np.array(xinput1,dtype=np.float32)
y1 = np.array(yinput1,dtype=np.float32)


# Define the neural network
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(5, 3)  # input layer -> hidden layer 1
        self.fc2 = nn.Linear(3, 2)  # hidden layer 1 -> hidden layer 2
        self.fc3 = nn.Linear(2, 1)  # hidden layer 2 -> output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define the loss function and optimizer
model = MLP()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Train the model
for epoch in range(1000):
    inputs = torch.from_numpy(x)
    targets = torch.from_numpy(y)
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, targets.unsqueeze(1))
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 1000, loss.item()))

# Test the model
with torch.no_grad():
    inputs = torch.from_numpy(np.array(xoutput1,dtype=np.float32))
    targets = torch.from_numpy(np.array(youtput1,dtype=np.float32))
    outputs = model(inputs)
    loss = criterion(outputs, targets.unsqueeze(1))
    print('Test loss: {:.4f}'.format(loss.item()))
    print('Predictions:', outputs.squeeze().numpy())

current, peak = tracemalloc.get_traced_memory()
print(f"Peak memory usage: {peak / 10**6} MB")

# Stop tracing memory usage
tracemalloc.stop()

Epoch [1/1000], Loss: 49.3037
Epoch [101/1000], Loss: 0.7157
Epoch [201/1000], Loss: 0.1360
Epoch [301/1000], Loss: 0.1258
Epoch [401/1000], Loss: 0.1256
Epoch [501/1000], Loss: 0.1256
Epoch [601/1000], Loss: 0.1256
Epoch [701/1000], Loss: 0.1256
Epoch [801/1000], Loss: 0.1256
Epoch [901/1000], Loss: 0.1256
Test loss: 4988.9536
Predictions: [0.14729334 0.14729334 0.14729334 0.14729334 0.14729334 0.14729334
 0.14729334 0.14729334 0.14729334 0.14729334 0.14729334 0.14729334
 0.14729334 0.14729334 0.14729334 0.14729334 0.14729334 0.14729334
 0.14729334 0.14729334 0.14729334 0.14729334 0.14729334 0.14729334
 0.14729334 0.14729334 0.14729334 0.14729334 0.14729334 0.14729334
 0.14729334 0.14729334 0.14729334 0.14729334 0.14729334 0.14729334
 0.14729334 0.14729334 0.14729334 0.14729334 0.14729334 0.14729334
 0.14729334 0.14729334 0.14729334 0.14729334 0.14729334 0.14729334
 0.14729334 0.14729334 0.14729334 0.14729334 0.14729334 0.14729334
 0.14729334 0.14729334 0.14729334 0.14729334 0.1472933

In [320]:
 labels

tensor([58.8000, 58.6000, 58.1000,  ..., 44.8000, 45.3000, 46.0000])